In [1]:
import tushare as ts
import pandas as pd
from datetime import datetime
import os.path
import time
import numpy as np

In [2]:
def update_data_or_not(fileName, year):
    import os
    yearEnd = datetime.now().year-1  #去年
    updateTime = 0
    if ((year==yearEnd and datetime.now().month >= 5) or year < yearEnd):
        updateTime = 30
    else:
        updateTime = 1
    updateTime = updateTime*24*60*60
    finfo = os.stat(fileName)
    return(( time.time() - finfo.st_mtime) > updateTime)
    

In [3]:
def get_data(dataType, year, q=4): #年报
    typeAll = {'profit', 'report', 'growth'}
    if (dataType not in typeAll):
        print('%s not in %s' %(type, typeAll,))
        return
    saveFileName = './data/%s%02d%s.csv' % (year,q, dataType)
    if os.path.isfile(saveFileName):
        df = pd.read_csv(saveFileName)#, encoding='gb18030')
        # return df
    if (update_data_or_not(saveFileName, year)):  # 如果需要更新
        fun = eval('ts.get_%s_data' % dataType)
        dfnew = fun(year, q)
        df = pd.concat([df[~df['code'].isin(dfnew['code'].values)],dfnew])
        df.to_csv(saveFileName, index=False, encoding='utf-8')
    return df


In [22]:
from pathlib import Path
year = '2020'
q = 4
dataType = 'profit'
saveFileName = Path('./data/%s%02d%s.csv' % (year,q, dataType))
saveFileName.exists()

False

In [44]:
def price_df(date, code):
    saveFileNameStr = './data/stock_%s_%s.csv' % (code,date)
    saveFileName = Path(saveFileNameStr)
    if (saveFileName.exists()):
        maxUpdateTime = 24*60*60
        finfo = saveFileName.stat()
        if ((time.time() - finfo.st_mtime) < maxUpdateTime):#更新时间小于1天
            df = pd.read_csv(saveFileName)
            return df
    df = ts.get_k_data(code, ktype='D', start=date)
    df.to_csv(saveFileNameStr, index=False, encoding='utf-8')
    return df
    

def price_after_report(date, code, index):
    return price_df(date, code).iloc[index]['close']



In [45]:
price_after_report('2017-01-25', '600816', 0)

11.08

In [52]:
stockGDF['price_after_report'] = stockGDF.apply(lambda x: \
                                                price_after_report(x['rec_report_date'], x['code'], 0)\
                                               , axis=1)

In [55]:
stockGDF['price_now'] = stockGDF.apply(lambda x: \
                                                price_after_report(x['rec_report_date'], x['code'], -1)\
                                               , axis=1)

In [56]:
stockGDF

,code,name0,mbrg0,nprg0,nav0,targ0,epsg0,seg0,name1,mbrg1,...,roe4,epcf4,net_profits4,profits_yoy4,distrib4,report_date4,eps,rec_report_date,price_after_report,price_now
0,300593,新雷能,32.2045,1554.8818,24.7699,22.1790,NaN,24.7699,新雷能,25.6332,...,NaN,NaN,NaN,NaN,NaN,NaN,0.51,2017-04-20,29.959,20.11
1,002614,奥佳华,25.8694,287.5262,4.0064,19.2742,295.4545,3.8546,奥佳华,26.0666,...,NaN,NaN,NaN,NaN,NaN,NaN,0.46,2017-04-25,16.708,18.96
2,603579,荣泰健康,25.9892,269.4559,-8.0437,27.7160,NaN,-8.3904,荣泰健康,75.7753,...,NaN,NaN,NaN,NaN,NaN,NaN,3.94,2017-02-27,59.265,65.51
3,601179,中国西电,4.5743,232.0633,25.2853,11.1603,159.2593,25.6805,中国西电,6.2670,...,NaN,NaN,NaN,NaN,NaN,NaN,0.22,2017-04-11,6.697,4.39
4,600867,通化东宝,21.4530,200.8781,1.2569,13.2066,150.0000,1.5026,通化东宝,20.5191,...,NaN,NaN,NaN,NaN,NaN,NaN,0.46,2017-04-12,17.459,22.19
5,002590,万安科技,25.7184,197.3318,5.4494,12.9761,190.0000,4.6133,万安科技,3.0472,...,NaN,NaN,NaN,NaN,NaN,NaN,0.27,2017-04-26,14.829,10.56
6,600816,安信信托,-100.0000,172.7590,37.1393,68.2672,159.6795,37.1393,安信信托,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.71,2017-01-25,11.080,13.06
7,300274,阳光电源,95.7290,148.6635,10.9263,41.8303,143.4783,10.9252,阳光电源,44.4187,...,NaN,NaN,NaN,NaN,NaN,NaN,0.41,2017-04-18,9.904,16.75
8,002126,银轮股份,13.6718,130.1681,8.7538,27.1628,130.7692,9.6411,银轮股份,26.1187,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,2017-04-15,9.211,9.05
9,300017,网宿科技,47.8880,128.5510,30.7636,41.6327,128.3582,30.7636,网宿科技,58.5711,...,NaN,NaN,NaN,NaN,NaN,NaN,1.62,2017-03-14,16.023,11.86


In [4]:
dfs = {'profit':{}, 'report':{}, 'growth':{}}
types = {'profit', 'report', 'growth'}
yearEnd = datetime.now().year  #今年
for t in types:
    for y in range(yearEnd-5, yearEnd):
        dfs[t][y] = get_data(t, y).rename(columns={i: i+str(y-yearEnd+5) for i in get_data(t, y).columns if i not in {'code'}})
        #dfs[t][y]['code'] = dfs[t][y]['code'].astype(int)
        dfs[t][y]['code'] = dfs[t][y]['code'].apply(lambda x: '%06d' % int(x))
#         
#             

In [5]:
dfs_growth = pd.DataFrame(columns=['code'])
for i in range(yearEnd-5, yearEnd):
    dfs_growth = dfs_growth.merge(dfs['growth'][i], on=['code'], how='outer')

# dfs['profit'][2013].merge(dfs['profit'][2014], on=['code', 'name'], how='outer')

In [24]:
stockGDF = dfs_growth[((np.isnan(dfs_growth.nprg4) & (dfs_growth.nprg0>25)) | (dfs_growth.nprg4>25))  &(dfs_growth.nprg1>25)\
           &(dfs_growth.nprg2>25)&(dfs_growth.nprg3>25)]

In [29]:
dfs_growth[[['nprg'+str(i) for i in range(5)]][((np.isnan(dfs_growth.nprg4) & (dfs_growth.nprg0>25)) | (dfs_growth.nprg4>25))]

KeyError: "['rec_report_date'] not in index"

In [6]:
stockGDF = dfs_growth[(dfs_growth.nprg0>25)&(dfs_growth.nprg1>25)\
           &(dfs_growth.nprg2>25)&(dfs_growth.nprg3>25)]


In [7]:
price = ts.get_realtime_quotes(stockGDF['code'])
def real_price(x, y):
    return x if x > 0 else y

price['price'] = price['price'].astype(float)
price['pre_close'] = price['pre_close'].astype(float)
price['price_real'] = price.apply(lambda x: real_price(x['price'], x['pre_close']), axis=1)
#price['price_true'] = map(lambda x, y : , price['price'].astype(float), price['pre_close'].astype(float))
#price = price.drop(['price_true'], axis=1)

In [8]:
stockGDF = stockGDF.merge(price, on='code')

In [48]:
dfs_report = pd.DataFrame(columns=['code'])
for i in range(yearEnd-2, yearEnd):
    dfs_report = dfs_report.merge(dfs['report'][i], on=['code'], how='outer')
# dfs['profit'][2013].merge(dfs['profit'][2014], on=['code', 'name'], how='outer')
dfs_report = dfs_report.drop_duplicates()
dfs_report['eps'] = dfs_report.apply(lambda x: x['eps3'] if pd.isnull(x['eps4']) else x['eps4'], axis=1)
dfs_report['rec_report_date'] = dfs_report.apply(lambda x: '%d-%s' % (yearEnd-1,x['report_date3'])\
                                                 if pd.isnull(x['report_date4']) else '%d-%s' % (yearEnd,x['report_date4']), axis=1)

In [49]:

stockGDF = stockGDF.merge(dfs_report, on='code', how='left')

In [11]:
#stockGDF[stockGDF.code == '002078']

In [62]:
stockGDF['nprg'] = stockGDF.apply(lambda x: x['nprg3'] if pd.isnull(x['nprg4']) else x['nprg4'], axis=1)

In [59]:
df_basics = ts.get_stock_basics()
stockGDF = stockGDF.merge(df_basics, left_on='code', right_index=True)

In [75]:
df_basics

,industry,area,pe,outstanding,totals,totalAssets,liquidAssets,fixedAssets,reserved,reservedPerShare,...,bvps,pb,timeToMarket,undp,perundp,rev,profit,gpr,npr,holders
code,,,,,,,,,,,,,,,,,,,,,
603506,区域地产,浙江,27.55,0.20,0.79,72642.00,0.00,0.00,0.00,0.00,...,4.35,5.38,20180201,0.00,0.00,0.00,0.00,0.00,8.71,18531.0
300739,元器件,深圳,31.85,0.31,1.23,96984.00,54019.57,36262.06,26068.90,2.12,...,5.45,5.89,20180201,14147.72,1.15,0.00,0.00,32.15,11.74,61548.0
300092,专用机械,四川,311.61,1.67,2.38,74803.03,51909.81,20203.23,26736.40,1.13,...,2.22,3.99,20100708,5963.39,0.25,-18.55,-62.30,20.06,2.47,18918.0
601838,银行,四川,10.68,3.61,36.12,40991660.00,0.00,102515.20,402383.19,1.11,...,7.68,1.44,20180131,1029206.69,2.85,0.00,0.00,0.00,43.06,299572.0
300277,软件服务,深圳,0.00,3.34,3.35,62416.96,56641.16,3643.01,8288.05,0.25,...,1.39,6.94,20111123,2956.40,0.09,28.20,79.86,16.95,-3.13,12806.0
603676,化学制药,西藏,79.89,0.63,4.23,93081.96,63901.90,15571.84,25683.25,0.61,...,1.97,9.86,20170721,12497.96,0.30,0.00,0.00,59.47,27.47,29886.0
300582,半导体,浙江,75.89,1.21,2.03,136238.97,45586.10,56919.20,55463.01,2.74,...,4.58,3.28,20161228,20920.55,1.03,18.08,-34.88,30.55,5.69,22339.0
002893,供气供热,北京,91.00,0.30,1.20,142167.38,70570.92,34371.30,26066.85,2.17,...,4.80,7.32,20170915,18700.89,1.56,0.00,0.00,17.46,6.33,58734.0
603058,广告包装,贵州,125.55,2.28,4.22,88269.03,42376.26,23044.42,13164.51,0.31,...,1.94,11.88,20161223,20059.55,0.48,-0.94,-13.59,35.62,26.19,7269.0


In [63]:
stockGDF['peg'] = round(stockGDF['pe']/stockGDF['nprg'],2)

In [15]:
stockGDF[['code', 'name','peg', 'pe', 'eps', 'nprg', 'price_real', 'rec_report_date']\
         + ['nprg'+str(i) for i in range(5)]][stockGDF.peg < 0.4]

,code,name,peg,pe,eps,nprg,price_real,rec_report_date,nprg0,nprg1,nprg2,nprg3,nprg4
6,600816,安信信托,0.21,15.80,1.71,76.1722,12.94,2017-01-25,172.7590,266.0663,68.2561,76.1722,NaN
10,002019,亿帆医药,0.24,21.07,0.64,88.8034,20.58,2017-03-31,126.2480,829.1151,52.1242,88.8034,NaN
23,300237,美晨生态,0.20,23.41,0.55,115.0827,13.11,2017-04-25,61.6673,237.5191,106.3751,115.0827,NaN
25,002841,视源股份,0.29,31.68,2.04,109.2521,63.53,2017-04-01,57.5867,37.6380,56.9934,109.2521,NaN
28,300244,迪安诊断,0.39,35.03,0.48,89.0256,20.79,2017-03-28,51.5895,42.0045,39.8372,89.0256,NaN
31,300253,卫宁健康,0.32,72.64,0.64,225.3914,8.45,2017-04-25,49.4068,56.4341,31.6772,225.3914,NaN
32,300253,卫宁健康,0.32,72.64,0.64,225.3914,8.45,2017-04-25,49.4068,56.4341,31.6772,225.3914,NaN
34,002078,太阳纸业,0.30,16.21,0.42,53.3837,11.35,2017-04-25,44.9830,72.0237,38.0570,53.3837,NaN
42,300296,利亚德,0.37,37.31,0.87,100.6575,19.66,2017-03-28,37.8114,104.1679,102.7194,100.6575,NaN
45,300121,阳谷华泰,0.08,22.87,0.56,298.7958,13.49,2017-02-24,34.3584,71.0576,45.7954,298.7958,NaN


In [69]:
stockGDF[['code', 'name0','peg', 'pe', 'eps', 'nprg', 'price_now', 'rec_report_date', 'price_after_report', '涨幅']\
         + ['nprg'+str(i) for i in range(5)]][stockGDF.peg < 0.4].describe()

,peg,pe,eps,nprg,price_now,price_after_report,涨幅,nprg0,nprg1,nprg2,nprg3,nprg4
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,0.0
mean,0.259231,31.605385,0.803846,131.851808,16.943846,21.107462,-0.082760,59.698415,144.164154,62.953977,131.851808,NaN
std,0.091784,19.434627,0.509600,75.430532,14.347163,22.997491,0.301151,42.317182,219.631397,31.026579,75.430532,NaN
min,0.080000,15.800000,0.390000,53.383700,6.240000,6.962000,-0.526124,26.554000,26.011300,31.677200,53.383700,NaN
25%,0.210000,20.390000,0.500000,85.387400,9.750000,10.086000,-0.266442,34.358400,42.004500,38.057000,85.387400,NaN
50%,0.290000,23.410000,0.640000,100.657500,13.060000,16.747000,-0.089572,49.406800,56.434100,52.124200,100.657500,NaN
75%,0.320000,35.030000,0.870000,177.655300,19.350000,18.043000,-0.028858,57.586700,104.167900,102.717200,177.655300,NaN
max,0.390000,72.640000,2.040000,298.795800,62.000000,94.570000,0.646079,172.759000,829.115100,108.406800,298.795800,NaN


In [68]:
stockGDF[['code', 'name0','peg', 'pe', 'eps', 'nprg', 'price_now', 'rec_report_date', 'price_after_report', '涨幅']\
         + ['nprg'+str(i) for i in range(5)]].describe()

,peg,pe,eps,nprg,price_now,price_after_report,涨幅,nprg0,nprg1,nprg2,nprg3,nprg4
count,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,0.0
mean,0.903333,45.157193,0.842456,66.736137,23.238596,21.424246,0.072415,102.822398,88.537825,72.883000,66.736137,NaN
std,0.673523,47.357487,0.666859,60.682295,18.677413,14.967091,0.522287,205.126483,117.717781,121.614073,60.682295,NaN
min,0.080000,15.800000,0.140000,25.695400,4.390000,6.697000,-0.526124,25.148200,26.011300,25.102700,25.695400,NaN
25%,0.440000,25.260000,0.390000,31.733100,10.560000,11.688000,-0.259814,37.811400,41.809400,33.765500,31.733100,NaN
50%,0.760000,35.430000,0.640000,45.895300,16.240000,17.112000,-0.089572,51.589500,54.117100,45.795400,45.895300,NaN
75%,1.080000,45.620000,1.220000,66.062500,28.000000,28.263000,0.218129,102.184500,72.023700,69.056300,66.062500,NaN
max,3.890000,355.270000,3.940000,298.795800,87.180000,94.570000,2.987915,1554.881800,829.115100,925.071200,298.795800,NaN


In [57]:
stockGDF['涨幅'] = (stockGDF['price_now'] - stockGDF['price_after_report'])/stockGDF['price_after_report']

In [74]:
stockGDF[['code', 'name0','peg', 'pe', 'eps', 'nprg', 'price_now', 'rec_report_date', 'price_after_report', '涨幅']\
         + ['nprg'+str(i) for i in range(5)]][stockGDF['涨幅']>0]

,code,name0,peg,pe,eps,nprg,price_now,rec_report_date,price_after_report,涨幅,nprg0,nprg1,nprg2,nprg3,nprg4
1,002614,奥佳华,0.83,37.94,0.46,45.8953,18.96,2017-04-25,16.708,0.134786,287.5262,32.8704,31.2568,45.8953,NaN
2,603579,荣泰健康,0.98,44.38,3.94,45.3725,65.51,2017-02-27,59.265,0.105374,269.4559,137.9635,131.6363,45.3725,NaN
4,600867,通化东宝,1.44,43.91,0.46,30.4331,22.19,2017-04-12,17.459,0.270978,200.8781,54.0926,77.2133,30.4331,NaN
6,600816,安信信托,0.21,15.80,1.71,76.1722,13.06,2017-01-25,11.080,0.178700,172.7590,266.0663,68.2561,76.1722,NaN
7,300274,阳光电源,0.86,24.23,0.41,28.1662,16.75,2017-04-18,9.904,0.691236,148.6635,56.4849,50.4169,28.1662,NaN
10,002019,亿帆医药,0.24,21.07,0.64,88.8034,20.40,2017-03-31,16.747,0.218129,126.2480,829.1151,52.1242,88.8034,NaN
15,002271,东方雨虹,0.73,29.74,1.22,40.8208,39.06,2017-04-21,28.578,0.366786,93.5067,57.3795,25.1027,40.8208,NaN
16,002271,东方雨虹,0.73,29.74,1.22,40.8208,39.06,2017-04-21,28.578,0.366786,93.5067,57.3795,25.1027,40.8208,NaN
20,300642,透景生命,1.04,50.70,2.17,48.6282,87.18,2017-04-10,51.814,0.682557,65.7828,59.4723,82.7018,48.6282,NaN
22,300630,普利制药,2.65,99.88,0.76,37.6250,66.00,2017-03-06,16.550,2.987915,61.7553,47.9018,38.2835,37.6250,NaN


In [ ]:
dfs_growth[['nprg'+str(i) for i in range(5)]][dfs_growth.code == '600522']

In [76]:
df_basics = ts.get_stock_basics(date='2018-01-01')

HTTPError: HTTP Error 404: Not Found

In [112]:
from datetime import datetime as dt
import datetime
import tushare as ts

def get_stock_basics(date):
    saveFileNameStr = './data/basics/%s.csv' % (date)
    saveFileName = Path(saveFileNameStr)
    if (saveFileName.exists()):
        df = pd.read_csv(saveFileName)
        df['code'] = df['code'].apply(lambda x: '%06d' % int(x))
        df.set_index('code', inplace=True, drop=True)
        return df
    try:
        df = ts.get_stock_basics(date)
        df.to_csv(saveFileNameStr, encoding='utf-8')
        return df
    except:
        tmpDate = dt.strptime(date, '%Y-%m-%d') + datetime.timedelta(days=1)
        date = tmpDate.strftime('%Y-%m-%d')
        df = get_stock_basics(date)
        df.to_csv(saveFileNameStr, encoding='utf-8')
        return df

def basics_df(date):
    if date is None:
        df = ts.get_stock_basics()
        return df
    df = get_stock_basics(date)
    return df
    
def pe(code, date=None):
    if code not in basics_df(date).index:
        return np.NaN
    return basics_df(date).loc[code, 'pe']


In [101]:
dftest = basics_df('2017-04-15')

In [89]:
dftest = pd.DataFrame([['300593', '2017-04-25']], columns=['code', 'rec_report_date'])

In [113]:
stockGDF['pe_report_date'] = stockGDF.apply(lambda x: pe(x['code'], x['rec_report_date']), axis=1)

In [96]:
stockGDF[stockGDF.code == '300642']['rec_report_date']

20    2017-04-10
Name: rec_report_date, dtype: object

In [107]:
stockGDF[stockGDF.code == '300642']['rec_report_date']['pe_report_date'] = np.NAN

C:\Users\jlgs-jz\Anaconda3\envs\rqalpha\lib\site-packages\pandas\core\series.py:747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
C:\Users\jlgs-jz\Anaconda3\envs\rqalpha\lib\site-packages\IPython\core\interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [120]:
# stockGDF.dropna(inplace=True)

In [121]:
stockGDF

,code,name0,mbrg0,nprg0,nav0,targ0,epsg0,seg0,name1,mbrg1,...,undp,perundp,rev,profit,gpr,npr,holders,nprg,peg,pe_report_date
